In [1]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from collections import Counter
from nltk.corpus import stopwords
from tqdm import tqdm
import fasttext.util
import pandas as pd
import numpy as np
import fasttext
import pickle
import string
import re

In [ ]:
from StemLemPipe import phrases2lower, phrases_without_excess_symbols, phrases_transform, text2sentences, split_by_words, sentence_split, create_stemmer_lemmer, words_to_ngrams_list, sum_phrases, wordlist2set, stopwords, StemLemPipeline

In [ ]:
import nltk
import spacy
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

In [2]:
#Выделение значений в скобках в отдельный столбец
#data['brackets'] = data.custom_position.apply(lambda value: re.findall('\(.*?\)', value))

#Удаление значений в скобках
#data['custom_position'] = data.custom_position.apply(lambda value: re.sub("\(.*?\)","()", value))

In [153]:
def spacy_process(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

def nltk_process(text):
    #Tokenization
    nltk_tokenList = word_tokenize(text)
    
    #Stemming
    nltk_stemedList = []
    for word in nltk_tokenList:
        nltk_stemedList.append(p_stemmer.stem(word))
    
    #Lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    return ' '.join([wordnet_lemmatizer.lemmatize(word) for word in nltk_stemedList])

def my_mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def prepare_data(data):
    data = data.drop(['id','salary_to','work_skills','skill_1','skill_2','skill_3','skill_4','skill_5','skill_6','skill_7','skill_8','skill_9','skill_10'], axis=1)
    
    top_5_city = data.city_id.value_counts().reset_index()['city_id'].values[:5].tolist()
    data.city_id = data.city_id.apply(lambda value: str(value) if value in top_5_city else 'Остальные')

    data.required_experience = data.required_experience.fillna('Отсутствует')
    
    enc_columns = ['schedule','education_name','city_id','required_experience']
    enc_data = pd.get_dummies(data[enc_columns], drop_first=True)
    
    data = data.drop(enc_columns, axis=1)
    data = pd.concat([data,enc_data], axis=1)

    data.custom_position = data.custom_position.str.lower().str.strip()
    return data

def get_emb_vector(custom_position, dim_size):
    
    ft = fasttext.load_model('cc.ru.300.bin')

    fasttext.util.reduce_model(ft, dim_size)
    
    vector_professions = []
    for words in custom_position:
        vector = np.mean([ft.get_word_vector(word) for word in words.split(' ')], axis=0).tolist()
        vector_professions.append(vector)
    return vector_professions

def get_emb_dataframe(custom_position, dim_size):
    
    ft = fasttext.load_model('cc.ru.300.bin')

    fasttext.util.reduce_model(ft, dim_size)
    
    vector_professions = []
    for words in custom_position:
        vector = np.mean([ft.get_word_vector(word) for word in words.split(' ')], axis=0).tolist()
        vector_professions.append(vector)
        
    df_vector_professions = pd.DataFrame(vector_professions, columns=[f"feature_{c}" for c in range(1, dim_size+1)])
    
    return df_vector_professions

def cv_fit_predict(data, fit_cols, text_col_name):
    
    data = data[fit_cols]
    
    df_vector_professions_300 = get_emb_dataframe(data[text_col_name].values.tolist(), 300)
    
    data = pd.concat([data, df_vector_professions_300], axis=1)
    
    y = data.salary_from
    X = data.drop(['salary_from', text_col_name], axis=1)
    
    num_folds = 5
    random_state = 42
    scoring = 'neg_mean_absolute_error'
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    model = DecisionTreeRegressor(random_state=random_state)
    
    kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)
    
    cv_results = -1 * cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    
    print([round(i,1) for i in cv_results])
    print(round(cv_results.mean(),2))

In [228]:
base_fit_cols = ['salary_from', 'count_skills',
       'schedule_полный рабочий день', 'schedule_свободный график',
       'schedule_сменный график', 'schedule_удаленная работа',
       'schedule_частичная занятость', 'education_name_высшее (бакалавр)',
       'education_name_любое', 'education_name_неполное высшее',
       'education_name_среднее', 'education_name_среднее профессиональное',
       'city_id_10', 'city_id_174', 'city_id_3', 'city_id_36',
       'city_id_Остальные', 'required_experience_Нет опыта',
       'required_experience_От 1 года до 3 лет',
       'required_experience_От 3 до 6 лет', 'required_experience_Отсутствует']

In [4]:
data = pd.read_csv('students_vacancies_10Nov2023.csv')

In [5]:
data.shape

(663547, 20)

In [6]:
%%time

data = prepare_data(data)

CPU times: total: 641 ms
Wall time: 649 ms


In [7]:
#Лучшее среднее значение MAE на 5 фолдах: 4335

cv_fit_predict(data, base_fit_cols+['custom_position'], text_col_name='custom_position')

[4396.8, 4328.4, 4312.8, 4378.2, 4258.4]
4334.93


In [8]:
#Замена знаков пунктуации
data['custom_position'] = data.custom_position.apply(lambda value: re.sub(r'[^\w\s]', ' ', value))

#Удаление лишних пробелов
data['custom_position'] = data.custom_position.apply(lambda value: re.sub('\s+', ' ', value).strip())

In [9]:
words  = []

for sublist in [i.split(' ') for i in data.custom_position.to_list()]:
    for item in sublist:
        words.append(item)
        
cnt_df = pd.DataFrame.from_dict(Counter(words), orient='index').reset_index()
cnt_df.sort_values(by=0, ascending=False).head(20)

,index,0
4,грузчик,232880
6,мерчендайзер,229047
1,кассир,184358
0,продавец,177933
5,старший,135251
2,сотрудник,119644
3,супермаркета,118817
12,администратор,86853
47,по,28274
19,водитель,19136


In [10]:
#Заменяем ка и ца
data['custom_position'] = data['custom_position'].replace({' ца ':' ', ' ка ':' '}, regex=True)

In [11]:
#Очищаем от стоп слов
stopwords = stopwords.words("russian")
data.custom_position = data.custom_position.apply(lambda value: ' '.join([word for word in value.split(' ') if word not in stopwords]))

In [65]:
example = data.custom_position.values.tolist()[:10000]

In [62]:
nlp = spacy.load('ru_core_news_sm')

In [66]:
%%time

spacy_example = [spacy_process(i) for i in example]

CPU times: total: 28.6 s
Wall time: 28.6 s


In [70]:
data.shape[0]/10000*28.6/60

31.629073666666667

In [76]:
%%time

data['spacy_custom_position'] = data.custom_position.apply(lambda value: spacy_process(value))

CPU times: total: 27min 32s
Wall time: 27min 33s


In [71]:
p_stemmer = PorterStemmer()

#s_stemmer = SnowballStemmer(language='english')

In [73]:
%%time

nltk_example = [nltk_process(i) for i in example]

CPU times: total: 797 ms
Wall time: 914 ms


In [74]:
data.shape[0]/10000*1/60

1.1059116666666666

In [79]:
%%time

data['nltk_custom_position'] = data.custom_position.apply(lambda value: nltk_process(value))

CPU times: total: 56.2 s
Wall time: 56.3 s


In [114]:
stem_lem = create_stemmer_lemmer(lemmatizer_backend='pymorphy', stemmer_backend='snowball')

In [115]:
data['stem_custom_position'] = phrases_transform(data.custom_position.to_list(), func = stem_lem)

In [81]:
data.to_csv('data_ext.csv')

In [83]:
data[data['spacy_custom_position']!=data['custom_position']][['custom_position','spacy_custom_position']]

,custom_position,spacy_custom_position
2,кассир сотрудник супермаркета,кассир сотрудник супермаркет
4,продавец старший сотрудник супермаркета,продавец старший сотрудник супермаркет
5,продавец старший сотрудник супермаркета,продавец старший сотрудник супермаркет
8,продавец сотрудник супермаркета,продавец сотрудник супермаркет
11,продавец старший сотрудник супермаркета,продавец старший сотрудник супермаркет
...,...,...
663540,продавец магазина магнит,продавец магазин магнит
663541,старший кассир старший сотрудник супермаркета,старший кассир старший сотрудник супермаркет
663544,менеджер продажам турагентство anex tour,менеджер продажа турагентство anex tour
663545,специалист приемке сотрудник супермаркета,специалист приёмка сотрудник супермаркет


In [82]:
data[data['nltk_custom_position']!=data['custom_position']][['custom_position','nltk_custom_position']]

,custom_position,nltk_custom_position
1486,помощник менеджера проектов junior project man...,помощник менеджера проектов junior project man...
4878,консультант бренда iqos,консультант бренда iqo
8974,менеджер интернет магазина online,менеджер интернет магазина onlin
11173,интернет маркетолог digital,интернет маркетолог digit
12576,менеджер работе маркетплейсами маркет озон wil...,менеджер работе маркетплейсами маркет озон wil...
...,...,...
662874,предлагаем удаленную работу вложений получайте...,предлагаем удаленную работу вложений получайте...
662889,предлагаем удаленную работу вложений получайте...,предлагаем удаленную работу вложений получайте...
662893,предлагаем удаленную работу вложений получайте...,предлагаем удаленную работу вложений получайте...
662901,предлагаем удаленную работу вложений получайте...,предлагаем удаленную работу вложений получайте...


In [101]:
text_col_name = 'spacy_custom_position'

cv_fit_predict(data, base_fit_cols+[text_col_name], text_col_name=text_col_name)

[4374.2, 4295.6, 4262.4, 4271.4, 4285.7]
4297.87


In [102]:
text_col_name = 'nltk_custom_position'

cv_fit_predict(data, base_fit_cols+[text_col_name], text_col_name=text_col_name)

[4331.8, 4305.8, 4278.8, 4283.6, 4268.3]
4293.65


In [116]:
text_col_name = 'stem_custom_position'

cv_fit_predict(data, base_fit_cols+[text_col_name], text_col_name=text_col_name)

[4357.5, 4326.9, 4270.3, 4284.1, 4310.4]
4309.85


### CatBoost

In [154]:
from catboost import Pool, CatBoostRegressor

In [155]:
emb = get_emb_vector(data['custom_position'].values.tolist(), 300)

In [156]:
ctb_data = data[base_fit_cols+['custom_position']]

In [158]:
ctb_data['emb'] = emb

C:\Users\User\AppData\Local\Temp\ipykernel_2728\252822995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctb_data['emb'] = emb


In [159]:
y = ctb_data.salary_from
X = ctb_data.drop(['salary_from', 'custom_position'], axis=1)

In [160]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [166]:
emb_features = ['emb']

In [172]:
cat_params = {
        'iterations': 1000,
        'early_stopping_rounds': 100,
        'loss_function': "MAPE",
        'random_seed': 42,
        'embedding_features': emb_features,
    }

In [173]:
train_pool = Pool(
                data=X_train,
                label=y_train,
                embedding_features=emb_features)

In [176]:
model = CatBoostRegressor(**cat_params)

In [177]:
model.fit(train_pool)

In [178]:
y_train_pred = model.predict(X_train)

print('train MAPE: ',mean_absolute_percentage_error(y_train, y_train_pred))

y_test_pred = model.predict(X_test)

print('test MAPE: ',mean_absolute_percentage_error(y_test, y_test_pred))

train MAPE:  0.18186194374273493
test MAPE:  0.1826064062388584


In [249]:
cnt = 21

my_data = cnt_df.sort_values(by=0, ascending=False)
my_data.columns=['word','cnt']

In [250]:
my_data['rank'] = my_data['cnt'].rank() #method='dense'
my_data['rank_perc'] = my_data['rank']/my_data.shape[0]*100

In [251]:
words = my_data[my_data['cnt']>5].word.to_list()

In [253]:
text_col_name = 'my_custom_position'

cv_fit_predict(data, base_fit_cols+[text_col_name], text_col_name=text_col_name)

[4282.8, 4281.0, 4224.6, 4249.3, 4242.0]
4255.95


In [254]:
words = my_data[my_data.rank_perc<99].word.to_list()

In [255]:
data['my_custom_position'] = data['custom_position'].apply(lambda value: ' '.join([i for i in value.split(' ') if i in words]))

In [256]:
text_col_name = 'my_custom_position'

cv_fit_predict(data, base_fit_cols+[text_col_name], text_col_name=text_col_name)

[6264.6, 6226.0, 6191.8, 6172.7, 6151.9]
6201.4
